# Syscall Reversing 
> Helper scripts for statically reversing malware with syscalls

- toc: true 
- badges: true
- categories: [Magniber,malware,research,syscalls,ransomware]

## Overview

When we are dealing with syscalls we need to be able to generate a syscall table from `ntdll.dll` that the sample is run with. The syscalls will vary between versions (DLL versions).

### Ransomware Example
- [Magniber](https://www.virustotal.com/gui/file/cb34e574d68c3646ea7985542b5385f134ff71b715ee10f364af43f63a992774/detection )
- [Threat Analysis Report: PrintNightmare and Magniber Ransomware](https://www.cybereason.com/blog/threat-analysis-report-printnightmare-and-magniber-ransomware)

In [15]:
import pefile
import struct
import re
pe = pefile.PE('/tmp/ntdll.dll')
pe_data = open('/tmp/ntdll.dll','rb').read()

In [12]:
pe.parse_data_directories(directories=[pefile.DIRECTORY_ENTRY['IMAGE_DIRECTORY_ENTRY_EXPORT']])
exports = []
for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
    export_address = exp.address
    export_name = exp.name
    export_ord = exp.ordinal
    exports.append({'name':export_name, 'ord':export_ord, 'address':export_address})


In [26]:
for export in exports:
    if b'ZwQueryEaFile' == export.get('name'):
        break
        
export_name = export.get('name').decode('utf-8')
export_offset = pe.get_offset_from_rva(export.get('address'))
print(f"{export_name} {hex(export_offset)}")

syscall_offset = pe_data.find(b'\x0f\05\xc3', export_offset)
print(f"Syscall offset: {hex(syscall_offset)}")
match = re.search(rb'\xB8(..)\x00\x00', pe_data[export_offset:syscall_offset])
if match:
    syscall_number = struct.unpack('<H', match.group(1))[0]
    print(f"Syscall: {hex(syscall_number)}")



ZwQueryEaFile 0xa1210
Syscall offset: 0xa1222
Syscall: 0x13e


In [36]:
syscalls = {}
for export in exports:
    if export.get('name') is not None and b'Zw' == export.get('name')[:2]:
        export_name = export.get('name').decode('utf-8')
        export_offset = pe.get_offset_from_rva(export.get('address'))
        syscall_offset = pe_data.find(b'\x0f\05\xc3', export_offset)
        if syscall_offset == -1:
            print(f"ERROR no sycall found for export {export_name}")
            continue
        match = re.search(rb'\xB8(..)\x00\x00', pe_data[export_offset:syscall_offset], re.DOTALL)
        if match is None:
            print(f"ERROR no sycall number for export {export_name}")
            continue
        syscall_number = struct.unpack('<H', match.group(1))[0]
        syscalls[export_name] = syscall_number

        
print("enum syscalls{")
for export in syscalls:
    print(f"sys_{export} = {syscalls[export]},")
print("};")

enum syscalls{
sys_ZwAcceptConnectPort = 2,
sys_ZwAccessCheck = 0,
sys_ZwAccessCheckAndAuditAlarm = 41,
sys_ZwAccessCheckByType = 99,
sys_ZwAccessCheckByTypeAndAuditAlarm = 89,
sys_ZwAccessCheckByTypeResultList = 100,
sys_ZwAccessCheckByTypeResultListAndAuditAlarm = 101,
sys_ZwAccessCheckByTypeResultListAndAuditAlarmByHandle = 102,
sys_ZwAcquireProcessActivityReference = 103,
sys_ZwAddAtom = 71,
sys_ZwAddAtomEx = 104,
sys_ZwAddBootEntry = 105,
sys_ZwAddDriverEntry = 106,
sys_ZwAdjustGroupsToken = 107,
sys_ZwAdjustPrivilegesToken = 65,
sys_ZwAdjustTokenClaimsAndDeviceGroups = 108,
sys_ZwAlertResumeThread = 109,
sys_ZwAlertThread = 110,
sys_ZwAlertThreadByThreadId = 111,
sys_ZwAllocateLocallyUniqueId = 112,
sys_ZwAllocateReserveObject = 113,
sys_ZwAllocateUserPhysicalPages = 114,
sys_ZwAllocateUuids = 115,
sys_ZwAllocateVirtualMemory = 24,
sys_ZwAllocateVirtualMemoryEx = 116,
sys_ZwAlpcAcceptConnectPort = 117,
sys_ZwAlpcCancelMessage = 118,
sys_ZwAlpcConnectPort = 119,
sys_ZwAlpcConnectP